In [ ]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torchvision
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torchvision.transforms import v2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.use_deterministic_algorithms(True) # Needed for reproducible results

In [ ]:
import os
import pandas as pd
import requests
from tqdm import tqdm

# 1) Load and clean URLs
df = pd.read_csv("pokemon-cards.csv")
urls = df["image_url"].dropna().unique()

# 2) Ensure output directory exists
os.makedirs("data", exist_ok=True)

# 3) Prepare a session for connection pooling
session = requests.Session()

# 4) Download loop with progress bar
for idx, url in enumerate(tqdm(urls, desc="Downloading images")):
    try:
        resp = session.get(url, timeout=10)
        resp.raise_for_status()
    except requests.RequestException as e:
        print(f"[{idx}] Failed to download {url!r}: {e}")
        continue

    # 5) Write to file with a context manager
    out_path = os.path.join("data", f"{idx}.jpg")
    with open(out_path, "wb") as f:
        f.write(resp.content)

In [ ]:
IMG_SIZE = 64
NUM_CHANELS = 3
BATCH_SIZE = 50

NUM_FEATURES = 64
LATENT_VECTOR_SIZE = 100

NUM_EPOCHS = 200

In [ ]:
data = torchvision.datasets.ImageFolder(
    root="./data",
    transform=v2.Compose([
        v2.ToImage(), 
        v2.ToDtype(torch.float32, scale=True),
        v2.Resize(IMG_SIZE),
        v2.CenterCrop(IMG_SIZE),
        v2.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]))

dataloader = torch.utils.data.DataLoader(
    dataset=data,
    batch_size=BATCH_SIZE,
    shuffle=True
    )

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.use_deterministic_algorithms(True) # Needed for reproducible results

In [ ]:
test_batch = next(iter(dataloader))

plt.figure(figsize=(8, 8))
plt.axis('off')
plt.imshow(np.transpose(torchvision.utils.make_grid(test_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))
plt.show()

In [ ]:
# out = (in - 1) x stride - 2 x padding + kernel_size + output_padding

class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            # initial input is Z 
            nn.ConvTranspose2d(
                in_channels=LATENT_VECTOR_SIZE, 
                out_channels=NUM_FEATURES * 8, 
                kernel_size=4, 
                stride=2, 
                padding=0,
                output_padding=0),
            nn.BatchNorm2d(num_features=NUM_FEATURES * 8),
            nn.ReLU(inplace=True),
            # after the above layer, the state size will be NUM_FEATURES x 8 x out x out = NUM_FEATURES x 8 x 4 x 4 = NUM_FEATURES x 8 x 4 x 4

            nn.ConvTranspose2d(
                in_channels=NUM_FEATURES * 8, 
                out_channels=NUM_FEATURES * 4, 
                kernel_size=4, 
                stride=2, 
                padding=1,
                output_padding=0),
            nn.BatchNorm2d(num_features=NUM_FEATURES * 4),
            nn.ReLU(inplace=True),
            # after the above layer, the state size will be NUM_FEATURES x 4 x out x out = NUM_FEATURES x 4 x 4 x 8 x 8


            nn.ConvTranspose2d(
                in_channels=NUM_FEATURES * 4, 
                out_channels=NUM_FEATURES * 2, 
                kernel_size=4, 
                stride=2, 
                padding=1,
                output_padding=0),
            nn.BatchNorm2d(num_features=NUM_FEATURES * 2),
            nn.ReLU(inplace=True),
            # after the above layer, the state size will be NUM_FEATURES x 2 x out x out = NUM_FEATURES x 2 x 2 x 16 x 16

            nn.ConvTranspose2d(
                in_channels=NUM_FEATURES * 2, 
                out_channels=NUM_FEATURES, 
                kernel_size=4, 
                stride=2, 
                padding=1,
                output_padding=0),
            nn.BatchNorm2d(num_features=NUM_FEATURES),
            nn.ReLU(inplace=True),
            # after the above layer, the state size will be NUM_FEATURES x out x out = NUM_FEATURES x 32 x 32

            nn.ConvTranspose2d(
                in_channels=NUM_FEATURES, 
                out_channels=NUM_CHANELS, 
                kernel_size=4, 
                stride=2, 
                padding=1,
                output_padding=0),
            nn.BatchNorm2d(num_features=NUM_CHANELS),
            nn.Tanh(),
            # after the above layer, the state size will be NUM_FEATURES x out x out = 3 x 64 x 64
        )
        
    def forward(self, input):
        return self.model(input)

In [ ]:
G = Generator().cuda()
print(G)

In [ ]:
# out = (in + 2 x padding - kernel_size) / stride + 1


class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(
                in_channels=3,
                out_channels=NUM_FEATURES,
                kernel_size=4,
                stride=2,
                padding=1,
            ),
            nn.BatchNorm2d(NUM_FEATURES),
            nn.LeakyReLU(),
            nn.Conv2d(
                in_channels=NUM_FEATURES,
                out_channels=NUM_FEATURES * 2,
                kernel_size=4,
                stride=2,
                padding=1,
            ),
            nn.BatchNorm2d(NUM_FEATURES * 2),
            nn.LeakyReLU(),
            nn.Conv2d(
                in_channels=NUM_FEATURES * 2,
                out_channels=NUM_FEATURES * 4,
                kernel_size=4,
                stride=2,
                padding=1,
            ),
            nn.BatchNorm2d(NUM_FEATURES * 4),
            nn.LeakyReLU(),
            nn.Conv2d(
                in_channels=NUM_FEATURES * 4,
                out_channels=NUM_FEATURES * 8,
                kernel_size=4,
                stride=2,
                padding=1,
            ),
            nn.BatchNorm2d(NUM_FEATURES * 8),
            nn.LeakyReLU(),
            nn.Conv2d(
                in_channels=NUM_FEATURES * 8,
                out_channels=1,
                kernel_size=4,
                stride=2,
                padding=0,
            ),
            nn.Sigmoid(),
        )

    def forward(self, input):
        return self.model(input)

In [ ]:
D = Discriminator().cuda()
print(D)

In [ ]:
loss_function = nn.BCELoss()
noise_batch = torch.rand(BATCH_SIZE, LATENT_VECTOR_SIZE, 1, 1, device=device)

real_label = 1
fake_label = 0

optimizerG = torch.optim.Adam(G.parameters(), lr=0.0002, betas=(0.4, 0.999))
optimizerD = torch.optim.Adam(D.parameters(), lr=0.0002, betas=(0.4, 0.999))

In [ ]:
noise = torch.rand(1, LATENT_VECTOR_SIZE, 1, 1, device=device)

with torch.no_grad():
    fake_images = G(noise)            # (1, nc, H, W)

generated_image = fake_images.cpu() * 0.5 + 0.5  # if you used Tanh

# 6. Plot
img = generated_image.squeeze(0).permute(1, 2, 0).clamp(0,1)  # HWC, [0,1]
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(NUM_EPOCHS):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):
        D.zero_grad()
        # Format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        # Forward pass real batch through D
        output = D(real_cpu).view(-1)
        # Calculate loss on all-real batch
        errD_real = loss_function(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, LATENT_VECTOR_SIZE, 1, 1, device=device)
        # Generate fake image batch with G
        fake = G(noise)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = D(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = loss_function(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        G.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = D(fake).view(-1)
        # Calculate G's loss based on this output
        errG = loss_function(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 2 == 0:
            print(
                "[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f"
                % (
                    epoch,
                    NUM_EPOCHS,
                    i,
                    len(dataloader),
                    errD.item(),
                    errG.item(),
                    D_x,
                    D_G_z1,
                    D_G_z2,
                )
            )

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 2 == 0) or (
            (epoch == NUM_EPOCHS - 1) and (i == len(dataloader) - 1)
        ):
            with torch.no_grad():
                fake = G(noise_batch).detach().cpu()
            img_list.append(
                torchvision.utils.make_grid(fake, padding=2, normalize=True)
            )

        iters += 1

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

In [ ]:
# Grab a batch of real images from the dataloader
real_batch = next(iter(dataloader))

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(torchvision.utils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()

In [ ]:
# torch.save(G.state_dict(), "./checkpoints/G1")
# torch.save(D.state_dict(), "./checkpoints/D1")

In [ ]:
%pip freeze > requirements.txt